In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install transformers
!pip install sentencepiece
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 145.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 104.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 70.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.0 

In [4]:
!pip install bertopic[visualization]

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-y8pe_h6p/kobert-tokenizer_e77a38c7d7804c17bedb2f3f9b5564b3
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-y8pe_h6p/kobert-tokenizer_e77a38c7d7804c17bedb2f3f9b5564b3
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=edf91cb63d4a8f79ca0296f3203228e04da76f8730d4348b03cd92d0cb675e70
  Stored in directory: /tmp/pip-ephem-wheel-cache-h4zzztsj/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [6]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [7]:
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']), attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape

torch.Size([1, 768])

In [8]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [9]:
inputs = tokenizer.batch_encode_plus([text], return_tensors='pt')
model_output = model(**inputs)
mean_pooling(model_output, inputs['attention_mask'])

tensor([[ 3.6122e-02, -1.2337e-01, -7.5301e-03, -5.0620e+00,  1.6959e-01,
          9.2553e-02, -2.8838e-01,  2.1198e-01, -6.3860e-01, -3.6451e-01,
         -1.6556e-02,  1.8412e-01, -2.9757e-01,  5.1233e-02,  4.4390e-02,
          2.3413e-01,  3.8918e-01, -2.7811e-02,  2.5196e-01, -1.6740e-01,
         -5.0612e-02, -1.1393e-01, -5.8496e-02, -9.2304e-02, -3.3941e-01,
          2.9963e-02, -6.9979e-02, -1.2118e-01, -1.7573e-01,  2.4531e-01,
          1.6764e-01,  6.0146e-02, -1.3626e-01,  1.6598e-01,  1.2201e-01,
          5.2110e-02, -3.8716e-02, -3.4097e-02,  2.7250e-01, -6.6029e-02,
          2.2798e-01,  2.6542e-02,  3.8524e-01, -5.3627e-02,  9.2007e-02,
          1.2872e-01, -9.4732e-02,  1.4018e-01,  4.4361e-02,  3.0786e-02,
         -2.6999e-01,  2.5637e-02,  5.1933e-02,  1.9702e-01, -1.4610e-01,
         -2.9428e-01,  1.8972e-01, -1.1122e-01, -2.5106e-01,  1.7953e-01,
          1.1555e-01, -1.8627e-01,  1.9638e-02, -1.5012e-01,  6.5611e-02,
          6.9550e-02,  1.3693e-01,  3.

In [10]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import os
import json

In [11]:
import re

cond = re.compile('[가-힣]+')

def get_data(gallery):
  result = []
  for name in os.listdir(path='/content/drive/MyDrive/jolnon/' + gallery):
    with open('/content/drive/MyDrive/jolnon/' + gallery + '/' + name, 'r', encoding='utf-8') as f:
      data = json.load(f)
      valid = []
      for d in data['content'].split('\n'):
        res = []
        for word in d.split(' '):
          if cond.match(word) != None:
            res.append(word)
        d = ' '.join(res)
        if d and not d.replace(' ', '').isdecimal():
          valid.append(d)
      result.append('\n'.join([data['title']] + valid))
  return result

In [12]:
import random
random.seed(42)

dataset = [get_data('cock_tail'), get_data('brandy'), get_data('ulisul'), get_data('wine')]
cocktail = []
for data in dataset:
  cocktail += data
cocktail[:5]

['조주기능사 빈자리 신청 가능할려나...\n이게 신청기한이 은근 빡세네...\n애초에 이런거 안봐서 몰랐음\n그래서 빈자리 신청 기한도 있는데\n오픈하자 마자 신청하면 가능할려나?',
 '그냥 그렇네요\n담에 마가리타나 해볼까',
 '타오바오 고수들 질문좀\n결제 하려고 누르면 저렇게 문제있다고 뜨면서 결제대기페이지 가라고 함\n근데 가서 또 결제 누르면 결제할 수 없는 상품이 있다고 뜨네\n결제할 카드입력 하는곳도 없고 왜케어렵노',
 '칵갤 티키쟁이들아\n머해먹음',
 '조주기능 필기교재 득템\n살까말까 하다가 학교 도서관에 있길래 빌려옴 종강하면 공부해봐야지']

In [14]:
def remove_words(sentence, word):
    return ' '.join([w for w in sentence.split(' ') if word not in w.lower()])
stopwords = ['ㅋㅋ', 'ex', '나는', 'on', '이미지', '순서', '오늘', '일단', 'and', '이야', '그리고', '내일', '그냥', '000', '조금', '살짝', 'ㅇㅇ', 'ㅈㄱㄴ', '있음', '이거', '내가', '칵하하하', '칵하핫', '근데', '지듣노', 'youtube ', '야스중', '우흥', '한다', 'ㅎㅎ', 'ㅠㅠ', '로오오오오오옹', '하고', '아침', '것도', '추천', '혹시', '새낀데', '같다']

In [15]:
for i in range(len(cocktail)):
  for j in range(len(stopwords)):
    cocktail[i] = remove_words(cocktail[i],stopwords[j])

In [16]:
class CustomTokenizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
  def __call__(self, target):
    return self.tokenizer.tokenize(target)

In [17]:
custom_tokenizer = CustomTokenizer(tokenizer)
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [18]:
# bertopic_model = BERTopic(embedding_model=model,
#                  vectorizer_model=vectorizer,
#                  nr_topics=50,
#                  top_n_words=10,
#                  calculate_probabilities=True)
bertopic_model = BERTopic(language='multilingual',
                          nr_topics=50,
                          top_n_words=10,
                          calculate_probabilities=True)

In [19]:
topics, probs = bertopic_model.fit_transform(cocktail)

In [20]:
bertopic_model.visualize_topics()

In [21]:
bertopic_model.visualize_distribution(probs[0])

In [22]:
for i in range(0, 50):
  print(i,'번째 토픽 :', bertopic_model.get_topic(i))

0 번째 토픽 : [('하이드아웃', 4.870364450322919), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]
1 번째 토픽 : [('러므코크', 1.115358577459456), ('금이레', 1.115358577459456), ('후라빵', 1.0163956354809232), ('신임', 1.0163956354809232), ('안티카', 0.9174907065177869), ('리키', 0.9174907065177869), ('팔레넘', 0.6749959122572445), ('', 1e-05), ('', 1e-05), ('', 1e-05)]
2 번째 토픽 : [('나눔릴', 0.3590583198050096), ('맨하탄', 0.33319250825839847), ('맨해튼', 0.30971318487407107), ('받고', 0.18559713742737224), ('브루클린', 0.16545975461317355), ('브롱스', 0.14935895222382578), ('라이', 0.11095553707701786), ('manhattan', 0.09078500049088596), ('transfer', 0.09078500049088596), ('내릴받고', 0.09078500049088596)]
3 번째 토픽 : [('얼음', 0.13402414774339058), ('탄산이', 0.05703727009268558), ('얼음을', 0.049834009930821335), ('더블스트레인', 0.04375898322862791), ('얼음에', 0.04375898322862791), ('스트레이너', 0.04188544529755114), ('얼린', 0.03093378021028897), ('긴얼음', 0.03093378021028897), ('벽면에', 0.030933

In [23]:
for i in range(8):
  print(topics.count(i))

29
23
20
22
17
14
15
28


In [24]:
print(topics)

[-1, 27, 15, -1, 33, -1, 40, -1, -1, -1, 22, 32, -1, -1, 0, 18, -1, -1, 28, -1, 21, 15, 12, 9, 30, -1, 27, 21, 37, 9, -1, 9, 30, 29, -1, -1, -1, -1, 15, 33, 9, 20, 37, -1, -1, -1, -1, -1, -1, 9, 12, -1, -1, -1, 6, -1, 6, 15, 6, 9, 30, 40, -1, 7, 1, -1, -1, -1, -1, 41, 27, -1, -1, 41, -1, -1, -1, 9, 6, -1, 30, 11, 1, 9, 30, 28, 30, 31, -1, -1, 40, 9, -1, 9, -1, 38, 9, -1, 28, -1, 39, 38, 15, 9, -1, 9, -1, 3, 32, 20, -1, -1, -1, 12, -1, 9, 21, 9, 37, 9, 5, -1, -1, 35, 16, -1, -1, -1, -1, 19, 9, -1, 30, -1, -1, 1, -1, -1, -1, -1, 9, 15, 37, 1, -1, 32, -1, 20, -1, -1, 9, 1, -1, 37, 9, 38, -1, 9, 9, 9, 38, 20, -1, 9, -1, -1, 9, 9, -1, 40, 40, -1, -1, 17, -1, 32, 27, 20, 20, 9, 12, -1, -1, 30, 9, 37, -1, 21, 41, -1, 40, -1, -1, -1, -1, -1, 7, 9, -1, -1, 40, 11, 30, -1, -1, -1, 14, -1, -1, 9, 30, 39, -1, -1, -1, 12, 11, 32, 37, -1, -1, 9, 9, 27, -1, 2, 26, 21, -1, 18, 16, -1, 36, 33, -1, -1, 30, 9, 9, 9, -1, 22, 37, -1, -1, -1, -1, 19, 25, -1, 14, -1, -1, -1, -1, -1, -1, 9, -1, 38, -1, -1, -1

In [25]:
from sklearn.cluster import KMeans

n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(probs)
labels = kmeans.labels_


In [26]:
real_labels = []
for i in range(len(dataset)):
  real_labels += [i] * len(dataset[i])

from sklearn.metrics import confusion_matrix
print(confusion_matrix(real_labels, labels))


[[ 276 2494   51   22]
 [  30  175    4    1]
 [  44  146    4    1]
 [ 371  645   93   14]]


In [27]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import confusion_matrix

def calc_purity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    _, col_ind = linear_sum_assignment(cm, maximize=True)
    reordered_cm = cm[:, col_ind]
    purity = np.sum(np.max(reordered_cm, axis=0)) / np.sum(reordered_cm)
    return purity, reordered_cm

purity, reordered_cm = calc_purity(real_labels, labels)

print("Purity: ", purity)
print("Reordered Confusion Matrix: \n", reordered_cm)

Purity:  0.6817661862274079
Reordered Confusion Matrix: 
 [[2494   51   22  276]
 [ 175    4    1   30]
 [ 146    4    1   44]
 [ 645   93   14  371]]
